In [1]:
import bs4 as bs
import requests
import pandas as pd


In [2]:
#Llamada a la página web. Se traen las tablas, cada una contine datos de 1 libro
#Dos columnas, la de la izquierda con el nombre de los datos (categorías), la derecha con los datos
#Cada fila es una lista con dos elementos ([0] categorías; [1] datos). 28 filas por tabla

data = []


for i in range(1,400):
    url = 'http://catalogo.bibliotecas.gob.ar/pergamo/opac/cgi-bin/pgopac.cgi?VDOC=1.'+str(i) + '#Ejemplares'
    resp = requests.get(url)
    
    resp = resp.text
    soup = bs.BeautifulSoup(resp, 'html.parser')
    table = soup.find('table', class_='wsTableGrid', id='tbGeneral')

    if table is None:
        continue
    for row in (table.find_all('tr')):
        derecha = row.find_all('td')
        data.append(derecha)     
      

In [3]:
len(data)

10005

In [4]:
#Creamos un diccionario para agrupar los datos según el nombre de cada uno. Y luego pasamos los datos agrupados a listas
#Resulta una lista (result) de 28 listas (cada una contiene los datos de las categorías de las tablas)
d = {}
for subl in data:
    try:
        if subl[0] in d:
            d[subl[0]].append(subl[1])
        else:
            d[subl[0]] = [subl[1]]
    except IndexError:
        continue
result = list(d.values())


In [5]:
len(d)

28

In [6]:
len(data)

10005

In [7]:
len(result)

28

In [9]:
#De las 399 iteraciones, quedan 299 puesto que en medio había listas vacías
len(result[4])

345

In [12]:
#Se crea una función para pasar los elementos de las listas de 'td' HTML a text
def pass_to_text(list_):
    text_list = []
    for tag in list_:
        text_list.append(tag.get_text())
     
    return(text_list)


In [13]:
#Se llama a la función para iterar sobre las listas con los datos que nos interesan
isbn = pass_to_text(result[0])
tipo_texto=pass_to_text(result[1])
titulo = pass_to_text(result[4])
autor = pass_to_text(result[7])
editorial = pass_to_text(result[10])
fecha_edicion = pass_to_text(result[11])
lugar_edicion = pass_to_text(result[13])
materia_0 = pass_to_text(result[26])
fecha_alta = pass_to_text(result[27])



In [14]:
# Limpiamos los strings de la lista materia_0 para que sólo quede el nombre de la materia
# sin perden elementos vacios

materia = []
for elem in materia_0:
    if elem:
        new_elem = ''.join(caracter
                           for caracter in elem
                           if caracter.isalpha() or
                           caracter.isspace() or
                           caracter == '-')
        materia.append(new_elem)
    else:
        materia.append(elem)


In [15]:
#Se crea un DataFrame con pandas
df = pd.DataFrame(list(zip(isbn,tipo_texto, titulo, autor, editorial, fecha_edicion, lugar_edicion, materia,fecha_alta)),
                  columns=['ISBN','Tipo_Texto', 'Título', 'Autor','Editorial','Fecha_Edición','Lugar_Edición','Materia','Fecha_Alta'])
df = df.reset_index(drop=True)
df


,ISBN,Tipo_Texto,Título,Autor,Editorial,Fecha_Edición,Lugar_Edición,Materia,Fecha_Alta
0,.,LITERATURA,Antología poética,"Marechal, Leopoldo (1900-1970); Andrés, Alfred...",Aguilar,1969,Buenos Aires,- LITERATURA ARGENTINA-POESIA,31/07/2001
1,950-515-107-1,LITERATURA,De mala muerte,"Abós, Álvaro (1941-...)",De la Flor,1986,Buenos Aires,- LITERATURA ARGENTINA-CUENTO,01/03/1996
2,84-261-0664-1,TEXTO NO LITERARIO,Confesiones,"Agustín, Santo (354-430); Esclassana, A. (trad.)",Juventud,1986,Barcelona,AGUSTIN FILOSOFIA-AGUSTIN SANTO -,01/03/1996
3,.,LITERATURA,El Capitán Veneno,"Alarcón, Pedro Antonio de (1833-1891)",Colihue,1985,Buenos Aires,- LITERATURA ESPAÑOLA-NOVELA,01/03/1996
4,.,LITERATURA,El sombrero de tres picos,"Alarcón, Pedro Antonio de (1833-1891)",Huemul,1971,Buenos Aires,- LITERATURA ESPAÑOLA-NOVELA,01/03/1996
...,...,...,...,...,...,...,...,...,...
340,950-13-2067-7,TEXTO NO LITERARIO,Nuevo castellano 1 : Lengua y literatura. Leng...,"Lacau, María Hortensia (1910-2006); Rosetti, M...",Kapelusz,1983,Buenos Aires,CASTELLANO-TEXTO SECUNDARIO LITERATURA -TEXTO...,12/03/1996
341,.,TEXTO NO LITERARIO,Nuevo castellano 2 : Lengua y literatura. Leng...,"Lacau, María Hortensia (1910-2006); Rosetti, M...",Kapelusz,1982,Buenos Aires,LENGUA -TEXTO SECUNDARIO LITERATURA -TEXTO SE...,12/03/1996
342,950-13-2069-3,TEXTO NO LITERARIO,Nuevo castellano 3 : Lengua y literatura. Leng...,"Lacau, María Hortensia (1910-2006); Rosetti, M...",Kapelusz,1982,Buenos Aires,LENGUA -TEXTO SECUNDARIO COMUNICACION-TEXTO S...,12/03/1996
343,950-46-0121-9,TEXTO NO LITERARIO,Ciencias biológicas 3,"Rubel, Diana Nora; Guahnon, Esther Liliana; La...",Santillana,1990,Buenos Aires,BIOLOGIA-TEXTO SECUNDARIO,12/03/1996


In [16]:
len(df)

345

In [52]:
#Se exportan los datos a archivos csv y xlsx

df.to_csv('D:\codoacodo 14\library_scraping\library_catalog.csv', index=True)

df.to_excel("D:\codoacodo 14\library_scraping\library_catalog.xlsx", index=False)
